# 0. Data Importation

In [1]:
import pandas as pd
import numpy as np

In [2]:
from Data_Engineering import load_all_matches

In [ ]:
#npm install odds-portal-scraper

In [ ]:
#npx odds-portal historic premier-league 2020 2023 --local ./data --odds-format eu

zsh:1: command not found: npx


In [3]:
df = load_all_matches("data")
df.to_parquet("all_odds.parquet", engine="fastparquet")
print(df.head())
print("Rows:", len(df))

                               match_id         date            home  \
0  09_Dec_2023_Crystal_Palace_Liverpool  09 Dec 2023  Crystal Palace   
1  09_Dec_2023_Crystal_Palace_Liverpool  09 Dec 2023  Crystal Palace   
2  09_Dec_2023_Crystal_Palace_Liverpool  09 Dec 2023  Crystal Palace   
3  09_Dec_2023_Crystal_Palace_Liverpool  09 Dec 2023  Crystal Palace   
4  09_Dec_2023_Crystal_Palace_Liverpool  09 Dec 2023  Crystal Palace   

        away       market   bookmaker outcome  odds  
0  Liverpool  ML_FullTime  Betclic.fr    home  6.75  
1  Liverpool  ML_FullTime  Betclic.fr    draw  4.60  
2  Liverpool  ML_FullTime  Betclic.fr    away  1.43  
3  Liverpool  ML_FullTime     bwin.fr    home  5.75  
4  Liverpool  ML_FullTime     bwin.fr    draw  4.20  
Rows: 39326


# 1. Full Time Arbitrage Detection

In [6]:
arb = []
group = df[df["market"]=="ML_FullTime"]

for match_id, subset in group.groupby("match_id"):

    # get best rows
    best_rows = (
        subset
        .sort_values("odds", ascending=False)
        .groupby("outcome")
        .head(1)
    )

    # extract values
    home_row = best_rows[best_rows["outcome"]=="home"].iloc[0]
    draw_row = best_rows[best_rows["outcome"]=="draw"].iloc[0]
    away_row = best_rows[best_rows["outcome"]=="away"].iloc[0]

    # odds
    home = home_row["odds"]
    draw = draw_row["odds"]
    away = away_row["odds"]
    
    #bookmakers
    home_book = home_row["bookmaker"]
    draw_book = draw_row["bookmaker"]
    away_book = away_row["bookmaker"]

    # arbitrage test
    if 1/home + 1/draw + 1/away < 1:
        print("Arbitrage found!", match_id)
        print(" Home:", home_book, home)
        print(" Draw:", draw_book, draw)
        print(" Away:", away_book, away)
        arb.append([match_id,home,draw,away,home_book,draw_book,away_book])

arbitrages_ft = pd.DataFrame(arb, columns=['Match','Home','Draw','Away','Home Book','Draw Book','Away Book'])

Arbitrage found! 02_Oct_2021_Chelsea_Southampton
 Home: bwin.fr 1.42
 Draw: Winamax 5.6
 Away: Winamax 11.0
Arbitrage found! 02_Oct_2021_Wolves_Newcastle
 Home: Betclic.fr 1.82
 Draw: Winamax 4.1
 Away: Winamax 5.0
Arbitrage found! 03_Oct_2021_Crystal_Palace_Leicester
 Home: Unibet.fr 3.25
 Draw: Winamax 3.5
 Away: Betclic.fr 2.68
Arbitrage found! 12_Aug_2023_Sheffield_Utd_Crystal_Palace
 Home: Unibet.fr 3.72
 Draw: Betclic.fr 3.2
 Away: Betclic.fr 2.42
Arbitrage found! 14_Aug_2021_Norwich_Liverpool
 Home: Betclic.fr 8.0
 Draw: Betclic.fr 5.2
 Away: Unibet.fr 1.5
Arbitrage found! 14_Aug_2023_Manchester_Utd_Wolves
 Home: Betclic.fr 1.36
 Draw: Unibet.fr 6.15
 Away: Unibet.fr 11.0
Arbitrage found! 15_May_2021_Southampton_Fulham
 Home: Betclic.fr 2.03
 Draw: Winamax 3.8
 Away: Winamax 4.25
Arbitrage found! 19_May_2021_Tottenham_Aston_Villa
 Home: Winamax 1.63
 Draw: Betclic.fr 4.9
 Away: Betclic.fr 5.9
Arbitrage found! 21_Apr_2021_Tottenham_Southampton
 Home: Betclic.fr 1.95
 Draw: Winama

In [7]:
arbitrages_ft.head()

,Match,Home,Draw,Away,Home Book,Draw Book,Away Book
0,02_Oct_2021_Chelsea_Southampton,1.42,5.6,11.00,bwin.fr,Winamax,Winamax
1,02_Oct_2021_Wolves_Newcastle,1.82,4.1,5.00,Betclic.fr,Winamax,Winamax
2,03_Oct_2021_Crystal_Palace_Leicester,3.25,3.5,2.68,Unibet.fr,Winamax,Betclic.fr
3,12_Aug_2023_Sheffield_Utd_Crystal_Palace,3.72,3.2,2.42,Unibet.fr,Betclic.fr,Betclic.fr
4,14_Aug_2021_Norwich_Liverpool,8.00,5.2,1.50,Betclic.fr,Betclic.fr,Unibet.fr
